In [1]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize
import string
from collections import Counter

In [2]:
# Download NLTK stopwords data
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nycdoe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nycdoe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nycdoe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
df=pd.read_csv('CL WordCloud.csv')

In [4]:
df['Summary By CI']= df.groupby('Change Idea')['Summary'].transform(lambda x: ' '.join(x))
df = df[['Change Idea','Summary By CI']].drop_duplicates().reset_index(drop=True)

In [5]:
df.head()

,Change Idea,Summary By CI
0,Change idea: 4- Collaborative Learning,The change idea implemented in this text is th...
1,Change idea: 5- Student Identities,The change idea being implemented is incorpora...
2,Change idea: 2- Language Demands,The change idea implemented by the F.I.R.E. te...
3,Change idea: 1- Academic Vocabulary,The change idea implemented in this text is te...


In [6]:
#df.to_csv('test.csv', index=False)

In [11]:
# Step 2: Define a function to generate WordCloud for a given text and save to file
def generate_and_save_wordcloud(text, filename,change_idea,title_fontsize=20, title_padding=6,dpi=800,n_gram=3, word_count=10):
    
    # Apply preprocessing to the text
    text = preprocess_text(text)
    
    
    # Generate verb_grams from processed text (e.g., between 2 to 4 words)
    min_words = 3
    max_words = 4
    # Generate n-grams from processed text
    #phrases = generate_verb_ngrams(text, n_gram)  # Change 3 to 4 for 4-word phrases, and so on
    phrases = generate_verb_grams_v2(text, min_words, max_words)
    
    
    # Generate n-grams from text
    #phrases = generate_ngrams(text, n_gram)  # Change 2 to 3 for 3-word phrases

    # Calculate frequencies
    #phrase_freq = {phrase: phrases.count(phrase) for phrase in set(phrases)}

    # Calculate frequencies
    phrase_freq = Counter(phrases)

    # Select the top 30 most frequent verb_phrases
    final_phrases = dict(phrase_freq.most_common(30))
    
    
    # wordcloud = WordCloud(width=400, height=400, background_color='white').generate(text)
    wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(final_phrases)
    
     # Create a subplot with adjustable spacing between title and wordcloud
    fig, ax = plt.subplots(figsize=(4, 4))
    plt.subplots_adjust(top=0.8 + title_padding/100)  # Adjust the top spacing
    
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_title(change_idea, fontsize=title_fontsize, pad=title_padding)  # Increase title fontsize and add padding
    ax.axis('off')
    
    
    
    plt.savefig(filename,dpi=dpi)
    plt.close()

# Function to generate n-grams from text
def generate_ngrams(text, n):
    words = text.split()
    ngrams = [tuple(words[i:i+n]) for i in range(len(words)-n+1)]
    return [' '.join(ngram) for ngram in ngrams]

# Function to remove stopwords and punctuation
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    translator = str.maketrans("", "", string.punctuation)
    
    # Tokenize and remove stopwords and punctuation
    tokens = [word.lower() for word in text.split() if word.lower() not in stop_words]
    tokens = [word.translate(translator) for word in tokens]
    
    return ' '.join(tokens)

# Function to check if a phrase contains a verb
def contains_verb(phrase):
    tags = pos_tag(word_tokenize(phrase))
    return any(tag.startswith('VB') for _, tag in tags)

# Function to generate n-grams from text containing a verb
def generate_verb_ngrams(text, n):
    words = text.split()
    ngrams = [tuple(words[i:i+n]) for i in range(len(words)-n+1) if contains_verb(' '.join(words[i:i+n]))]
    return [' '.join(ngram) for ngram in ngrams]


# Function to generate verb_grams from text with a specified word range
def generate_verb_grams_v2(text, min_words, max_words):
    words = text.split()
    verb_grams = [
        tuple(words[i:i+n]) 
        for n in range(min_words, max_words + 1) 
        for i in range(len(words)-n+1) 
        if contains_verb(' '.join(words[i:i+n]))
    ]
    return [' '.join(gram) for gram in verb_grams]

# Step 3: Iterate through groups and generate WordCloud for each 'grouped_summary'
for change_idea, group_df in df.groupby('Change Idea'):
    grouped_summary_text = ' '.join(group_df['Summary By CI'])

    change_idea = change_idea.replace("Change idea: ", "")
    filename = f'wordcloud_{change_idea}.png'  # Adjust the filename as needed
    print(filename)
    
    generate_and_save_wordcloud(grouped_summary_text, filename, change_idea)
    
    #wordcloud = WordCloud(width=800, height=400, background_color='white').generate(grouped_summary_text)
    #plt.figure(figsize=(10, 5))
    #plt.imshow(wordcloud, interpolation='bilinear')
    #plt.axis('off')
    #plt.show()

wordcloud_1- Academic Vocabulary.png
wordcloud_2- Language Demands.png
wordcloud_4- Collaborative Learning.png
wordcloud_5- Student Identities.png
